In [1]:
import numpy as np
import torchvision
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from skimage import io
import torch
import torch.nn as nn
from einops import rearrange, repeat
import pandas as pd
import sys
sys.path.insert(0, '../utils/')
from dataset import ChestImage64
import os
import time
import datetime

import matplotlib.pyplot as plt
import csv

from torchvision.models import vit_b_16, ViT_B_16_Weights
from torch.utils.data import random_split, DataLoader, Dataset

In [2]:
# csv_path = "../64pxImages/train_labels_64p.csv"
# root_path = '../64pxImages'

csv_path = "../Data/256pxImages/train_labels_256p.csv" #I had to change \ to / for this but disregared if it is not an issue for you
root_path = "../Data/256pxImages/"


default_transform = ViT_B_16_Weights.IMAGENET1K_SWAG_LINEAR_V1.transforms


data_transform = Compose([
    Resize((64, 64)),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [3]:
myCSV = pd.read_csv(csv_path)
myCSV['EncodedLabels'] = ''
print(myCSV.shape)


# for i in range(4, myCSV.shape[1]-1):
#     myCSV['EncodedLabels'] = myCSV['EncodedLabels'].astype(str) + myCSV.iloc[:, i].astype(str) 
#     if i < myCSV.shape[1]-2:
#         myCSV['EncodedLabels'] = myCSV['EncodedLabels'].astype(str) + "," 

for i in range(4, myCSV.shape[1]-1):
    myCSV['EncodedLabels'] = myCSV['EncodedLabels'].astype(str) + myCSV.iloc[:, i].astype(str) 
    if i < myCSV.shape[1]-2:
        myCSV['EncodedLabels'] = myCSV['EncodedLabels'].astype(str) + "," 



# myCSV['EncodedLabels'] = myCSV['EncodedLabels'].astype(str) + "]"


# We can use the encodedlabels column as our labels for our data

# since we are not useing cross attention, pull out only the frontal images. 
frontalCSV = myCSV[myCSV['Frontal/Lateral'].str.contains("Frontal")]
frontalCSV.head()

filename = frontalCSV.iloc[1, 0]
print(filename)

label_test = frontalCSV['EncodedLabels'].iloc[0]

test_path = os.path.join(root_path, filename)


label_test = [int(x) for x in label_test.split(",")]

print("label_test: ", label_test)

image = io.imread(test_path)
print(type(image))
image = torch.tensor(image)
print(image.size())

(61266, 19)
Frontal/patient00004_study1_Frontal.png
label_test:  [1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0]
<class 'numpy.ndarray'>
torch.Size([256, 256])


In [4]:
myCSV.head()

,256path,Patient,Study,Frontal/Lateral,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,No Finding,EncodedLabels
0,Frontal/patient00002_study1_Frontal.png,patient00002,study1,Frontal,1,1,1,1,0,1,1,1,0,0,1,1,0,0,"1,1,1,1,0,1,1,1,0,0,1,1,0,0"
1,Lateral/patient00002_study1_Lateral.png,patient00002,study1,Lateral,1,1,1,1,0,1,1,1,0,0,1,1,0,0,"1,1,1,1,0,1,1,1,0,0,1,1,0,0"
2,Frontal/patient00004_study1_Frontal.png,patient00004,study1,Frontal,0,0,0,0,0,0,0,0,0,0,0,0,0,1,"0,0,0,0,0,0,0,0,0,0,0,0,0,1"
3,Lateral/patient00004_study1_Lateral.png,patient00004,study1,Lateral,0,0,0,0,0,0,0,0,0,0,0,0,0,1,"0,0,0,0,0,0,0,0,0,0,0,0,0,1"
4,Frontal/patient00005_study1_Frontal.png,patient00005,study1,Frontal,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"0,0,0,0,0,0,0,0,0,0,0,0,1,0"


In [5]:
# load up the dataset
class CustomDataset(Dataset):
    def __init__(self, df, root_dir, label_col, transform = None):
        self.df = df
        self.root_dir = root_dir
        self.label_col = label_col
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):

        if torch.is_tensor(index):
            index = index.tolist()

        # get the filename of the image
        filename = self.df.iloc[index, 0]
        label = self.df[self.label_col].iloc[index]

        if type(label) == str:
            label = [int(x) for x in label.split(",")]

        # load the image from disk
        path = os.path.join(self.root_dir, filename)
        img = io.imread(path)



        label = torch.tensor(label)
        label = label.float()
        img = torch.tensor(img)
        img = img.resize_((224, 224))
        img = repeat(img, "h w -> (repeat h) w", repeat = 3)
        img = rearrange(img, "(c h) w -> c h w", c = 3)
        img = img.float()


        # if self.transform:
            # label = self.transform(label)
            # img = self.transform(img)

        # return the image and its filename
        return img, label


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [7]:
def train_model(train_dataset, validation_dataset,config, benchmark=0.33):
    #, epochs=1
    start_time = time.time()
    
    learning_rate=config["lr"]
    batch_size=config["batch_size"]
    #dropout=config["d1"]
    epochs=config["epochs"]
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset,batch_size=batch_size, shuffle=True)
    
    
    # information about the pretrained models is coming from this link: 
    #https://pytorch.org/vision/master/models.html


    # just use the default weights. These should yeild the best results
    weights = ViT_B_16_Weights.DEFAULT
    num_classes = 14
    feature_extraction = False
    model = vit_b_16(weights = weights)
    
    criterion = nn.BCEWithLogitsLoss()
    
    if feature_extraction: 
        for param in model.parameters():
            param.requires_grad = False

        # change the last layer to have the correct number of classes
        model.heads = nn.Sequential(nn.Linear(768, num_classes))
        model.heads.requires_grad_ = True
    else:
        model.heads = nn.Sequential(nn.Linear(768, num_classes))

    
    model.to(device)

    
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9) 
    
    training_losses = []
    validation_losses = []
    validation_accuracies =[]
    ####
    #prev_epoch_accuracy=0.00001 #Just above zero
    ####
    for epoch in range(epochs):
        phase = 'train'
        # set the model to training mode
        model.train()
        running_loss = 0.0
    
        for i, data in enumerate(train_loader):
            inputs, labels = data[0].to(device), data[1].to(device)

            # ugh. This is gross. I should have done this step at the beginning for all of the datasets. 
            # zero the parameter gradients
            optimizer.zero_grad()
            # print("Here's the size of the inputs: ", inputs.size())
            # with torch.set_grad_enabled(phase == 'train'):
                # run the training data through the model
            outputs = model(inputs)

            #calculate the loss of the model
            loss = criterion(outputs, labels)

            running_loss += loss.item()
        
        
            if i % 100 == 99:    # record loss and test validation set
                model.eval()
                v_running_loss = 0.0
                
                ####
                #total = 0
                #correct = 0
                #accuracy=0
                #####
                
                for v, data in enumerate(val_loader):
                    inputs, labels = data[0].to(device), data[1].to(device)
                    v_outputs = model(inputs)
                    v_loss = criterion(v_outputs, labels)
                    v_running_loss += v_loss.item()
                    
                    ####
                    #_, predicted = torch.max(v_outputs.data,1)
                    #total += labels.size(0)
                    #correct += (predicted == labels).sum().item()

                    #vloss = criterion(v_outputs,labels)
                    #validation_running_loss += vloss.item()
                    ####
                ####
                #accuracy = correct/total
                #accuracy == (true == mdl(x).max(1).item()) / true.size(0)
                ####
                
                print("Time: ", datetime.datetime.now().strftime("%H:%M:%S"), "\tepoch: ", epoch+1, "batch: ", i+1, "Training loss: ", running_loss, "Validation loss ", v_running_loss)
                #print("Time: ", datetime.datetime.now().strftime("%H:%M:%S"), "\tepoch: ", epoch+1, "batch: ", i+1, "Training loss: ", running_loss, "Validation loss ", v_running_loss, "Validation accuracy: ", accuracy)
                validation_losses.append(v_running_loss)
                training_losses.append(running_loss)
                running_loss = 0.0
                
                ####
                #If first epoch accuracy is less than the benchmark, then break
                #if accuracy < benchmark:
                #    break
                #
                #If less than a 1% improvement in accuracy, terminate early.
                #epoch_acc_change = (accuracy - prev_epoch_accuracy)/prev_epoch_accuracy
                #if epoch_acc_change < 0.01:
                #    break
                #prev_epoch_accuracy = accuracy
                ####

            # once the validation has been completed, update the model
            loss.backward()
            optimizer.step()
            # set model back to training mode

    end_time = time.time()
    train_time = end_time - start_time
    print("Elapsed Training Time: ", datetime.timedelta(seconds = train_time))
    print('Finished Training')
    
    #model_name= "vitb_model_e" + str(epochs) + f"_drop{dropout:.0e}" + f"_lr{learning_rate:.0e}" + "_bs "+ str(batch_size)
    model_name= "vitb_model_e" + str(epochs) + f"_lr{learning_rate:.0e}" + "_bs "+ str(batch_size)
    model_filepath="../Logging/" + model_name + ".pth"

    figure_path="../Logging/" + model_name + ".png"

    print(model_filepath)

    torch.save(model.state_dict(), model_filepath)

    fig, axs = plt.subplots()
    axs.plot(training_losses)
    axs.plot(validation_losses)
    axs.set_title("Losses")
    axs.set_xlabel("Epochs")
    axs.set_ylabel("Loss")

    fig.suptitle(model_name)
    plt.savefig(figure_path, dpi=300)
    
    #return running_loss, v_running_loss, accuracy, epoch
    return running_loss, v_running_loss, epoch
    

In [8]:
dataset = CustomDataset(frontalCSV, root_dir=root_path, label_col="EncodedLabels", transform=default_transform)

#print(type(dataset))

# split into test train validate
train_size = int(0.7 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = int(0.2 * len(dataset))


train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

print("Train Length: ", len(train_dataset))
print("Validation Length: ", len(val_dataset))
print("Test Length: ", len(test_dataset))

#train_loader = DataLoader(train_dataset, batch_size=batchsize, shuffle=True)
#val_loader = DataLoader(val_dataset, batch_size=batchsize, shuffle=False)
#test_loader = DataLoader(test_dataset,batch_size=batchsize, shuffle=True)

Train Length:  21441
Validation Length:  3063
Test Length:  6126


In [ ]:
result_list = []

#Grid search parameters
#dropout=[0.2]
batch_size=[2,4,8,12]
learn_rate=[1,0.1,0.01,0.001]
epochs=[1,2,3]


it=0
#for d in dropout:
for e in epochs:
    for bs in batch_size:
        for lr in learn_rate:

            it +=1 
            #config = {"d1": d,"lr": lr,"batch_size": bs}
            config = {"epochs": e, "lr": lr, "batch_size": bs}
            #config = {"lr": lr,"batch_size": bs}

            print(config)
            print("Train Start. Iteration: ",it)
            t_loss,v_loss, end_Ep=train_model(train_dataset, val_dataset,config)
            #t_loss,v_loss, acc, end_Ep=train_model(train_dataset, val_dataset,config)

            list_row= str(bs) + "," + str(lr) + "," + str(t_loss) + "," + str(v_loss) + "," + str(end_Ep)
            #list_row= str(bs) + "," + str(lr) + "," + str(t_loss) + "," + str(v_loss) + "," + str(end_Ep), + "," + str(acc)

            t_list=[list_row]  
            result_list.append(t_list)


with open('opt_resultss.csv','w') as result_file:
    wr = csv.writer(result_file, dialect='excel')
    wr.writerow(result_list)

{'epochs': 1, 'lr': 1, 'batch_size': 2}
Train Start. Iteration:  1
Time:  17:48:19 	epoch:  1 batch:  100 Training loss:  nan Validation loss  nan
Time:  17:48:40 	epoch:  1 batch:  200 Training loss:  nan Validation loss  nan
Time:  17:49:02 	epoch:  1 batch:  300 Training loss:  nan Validation loss  nan
Time:  17:49:23 	epoch:  1 batch:  400 Training loss:  nan Validation loss  nan
Time:  17:49:44 	epoch:  1 batch:  500 Training loss:  nan Validation loss  nan
Time:  17:50:05 	epoch:  1 batch:  600 Training loss:  nan Validation loss  nan
Time:  17:50:26 	epoch:  1 batch:  700 Training loss:  nan Validation loss  nan
Time:  17:50:47 	epoch:  1 batch:  800 Training loss:  nan Validation loss  nan
Time:  17:51:08 	epoch:  1 batch:  900 Training loss:  nan Validation loss  nan
Time:  17:51:29 	epoch:  1 batch:  1000 Training loss:  nan Validation loss  nan
Time:  17:51:50 	epoch:  1 batch:  1100 Training loss:  nan Validation loss  nan
Time:  17:52:11 	epoch:  1 batch:  1200 Training lo